In [1]:
from pydub.exceptions import CouldntDecodeError
import os
from pydub import AudioSegment
import pandas as pd

import csv

import json

In [ ]:
rootdir = '../EMBRACE-data-analysis/data/summer23-pitt'

df_duration_audios = []#pd.DataFrame()

current_family = ""
total_time_duration = 0
total_chapter_openings = 0
for subdir, dirs, files in os.walk(rootdir):

    if "Family_" in subdir:
        print(subdir)
        if current_family=="" or current_family not in subdir:
            if(current_family!=""):
                df_duration_audios.append({"CCID":current_family,"duration":total_time_duration,"chapter_openings":total_chapter_openings})
                print(current_family+" total time: "+str(total_time_duration)+" chapter openings: "+str(total_chapter_openings))
            index_family = subdir.find("Family_")
            current_family = subdir[index_family:index_family+10]
            print("New family: "+current_family)
            total_time_duration = 0
            total_chapter_openings = 0
    for file in files:
        file_path = os.path.join(subdir, file)

        if file_path.endswith(".mp3"):
            file_stats = os.stat(file_path)
            file_size = file_stats.st_size
            if(file_size>2000):
                #print(file_path)
                #print(f'File Size in Bytes is {file_size}')
                try:
                    try:
                        audio = AudioSegment.from_file(file_path, "mp3")
                    except:
                        audio = AudioSegment.from_file(file_path, format="mp4")

                    total_time_duration = total_time_duration + audio.duration_seconds
                    total_chapter_openings = total_chapter_openings + 1
                    #print(subdir+" -> total time duration: "+str(total_time_duration)+" seconds")
                except CouldntDecodeError:
                    print("Error on reading the file: "+file_path)

#    if(total_time_duration>0):
#        df_duration_audios.append(total_time_duration)

#Last family's data
df_duration_audios.append({"CCID":current_family,"duration":total_time_duration,"chapter_openings":total_chapter_openings})
print(current_family+" total time: "+str(total_time_duration)+" chapter openings: "+str(total_chapter_openings))

header = ["CCID","duration","chapter_openings"]
with open('summary-pittsummer23-audios.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    for row in df_duration_audios:
        writer.writerow(row)
#show stats about total duration of the audios per family
# sum_durations = 0
# total_families = 0
# for i in range(0,len(df_duration_audios)):
#     print(df_duration_audios[i])
#     total_families = total_families + 1
#     sum_durations = sum_durations + df_duration_audios[i]
# print("average of total duration is: ")
#print(sum_durations/total_families)

#print(df_duration_audios.describe())



In [42]:
rootdir = '../EMBRACE-data-analysis/data/summer23-pitt'

list_act_logs = []#pd.DataFrame()

current_family = ""

for subdir, dirs, files in os.walk(rootdir):

    if "Family_" in subdir:
        print(subdir)
        if current_family=="" or current_family not in subdir:
            #if(current_family!=""):
                #df_act_logs.append({"CCID":current_family,"data":0})
            index_family = subdir.find("Family_")
            current_family = subdir[index_family:index_family+10]
            print("New family: "+current_family)

    for file in files:
        file_path = os.path.join(subdir, file)

        if file_path.endswith("systemactions.json"):
            print("Found the json!")

            # Opening JSON file
            f = open(file_path)

            # returns JSON object as
            # a dictionary
            data = json.load(f)

            # Iterating through the json
            # list
            for row in data:
                if row['actionName']=='clickPrompt':
                    print(row)
                    row['CCID']=current_family
                    list_act_logs.append(row)

            # Closing file
            f.close()

# Each element is a dict
d_series = pd.Series(list_act_logs)

df_act_logs = pd.DataFrame.from_records(d_series)

print(df_act_logs.loc[[20]]['actionDescription'])


../EMBRACE-data-analysis/data/summer23-pitt/Family_34
New family: Family_34
../EMBRACE-data-analysis/data/summer23-pitt/Family_34/34_Chromebook Data
../EMBRACE-data-analysis/data/summer23-pitt/Family_34/34_Chromebook Data/34_voice-recordings
../EMBRACE-data-analysis/data/summer23-pitt/Family_34/34_Chromebook Data/34_data_json
Found the json!
{'_id': {'$oid': '6477ca939544917a15c18f01'}, 'user': 'cc107', 'dateTime': {'$date': '2023-05-31T22:30:43.093Z'}, 'actionName': 'clickPrompt', 'actionDescription': 'Tell your child:,The names of some animals that are on the farm->off', 'isAsync': True, 'actionDuration': 0, 'bookLanguage': 'English (EN)', 'bookId': 'The Best Farm (pre-schoolers)', 'chapterId': 'The Contest', 'currPageID': 1, 'currentSentenceID': 2, '__v': 0}
{'_id': {'$oid': '6477ca9b9544917a15c18f03'}, 'user': 'cc107', 'dateTime': {'$date': '2023-05-31T22:30:51.301Z'}, 'actionName': 'clickPrompt', 'actionDescription': 'Tell your child:,The names of some animals that are on the farm

In [48]:
#Aggregation of the data

# Using groupby() and count()
# to get the count of each group
#df_off filter
df_prompts_off= df_act_logs[df_act_logs['actionDescription'].str.endswith('->off')]
#Unique clicks
#df_prompts = df_prompts_off.groupby(['CCID'])['actionDescription'].nunique().reset_index(name="clickPrompts")

#Total number of clicks (repeated)
df_prompts = df_prompts_off.groupby(['CCID'])['actionDescription'].count().reset_index(name="clickPrompts")

print("Get count of each family:\n", df_prompts)

header = ["CCID","clickPrompts"]
with open('summary-pittsummer23-prompts.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    dict_prompts = df_prompts.to_dict(orient='records')
    for row in dict_prompts:
        writer.writerow(row)


Get count of each family:
          CCID  clickPrompts
0   Family_01           138
1   Family_02             5
2   Family_03            46
3   Family_04            84
4   Family_05             2
5   Family_06            75
6   Family_07            72
7   Family_08           242
8   Family_09            75
9   Family_10            94
10  Family_11           106
11  Family_12           214
12  Family_13           229
13  Family_14           217
14  Family_15           143
15  Family_17            29
16  Family_18           109
17  Family_19           101
18  Family_20            80
19  Family_21            91
20  Family_22            37
21  Family_23           241
22  Family_24           316
23  Family_25           102
24  Family_26            79
25  Family_27           224
26  Family_28           175
27  Family_29           266
28  Family_31           207
29  Family_32            96
30  Family_33            47
31  Family_34             5
32  Family_35            22
33  Family_36        